In [28]:
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.chroma import Chroma
from langchain.llms import OpenAI
from langchain import PromptTemplate
import torch
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import numpy as np

In [5]:
from dotenv import load_dotenv, find_dotenv
dotenv = 'test_keys.env'
load_dotenv(dotenv)

True

In [30]:
%pip install -q ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [31]:
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin', callbacks=[StreamingStdOutCallbackHandler()])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat     │
│   2                                                                                              │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/langchain_core/load/ser │
│ ializable.py:97 in __init__                                                                      │
│                                                                                                  │
│    94 │   _lc_kwargs = PrivateAttr(default_factory=dict)                                         │
│    95 │                                                                                          │
│    96 │   def __init__(self, **kwargs: Any) -> None:                                             │
│ ❱  97 │   │   super().__init__(**kwargs)                                                         │
│    98 │   │   self._lc_kwargs = kwargs                                                           │
│    99 │                                                                                          │
│   100 │   def to_json(self) -> Union[SerializedConstructor, SerializedNotImplemented]:           │
│                                                                                                  │
│ in pydantic.main.BaseModel.__init__:339                                                          │
│                                                                                                  │
│ in pydantic.main.validate_model:1102                                                             │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/langchain/llms/ctransfo │
│ rmers.py:73 in validate_environment                                                              │
│                                                                                                  │
│    70 │   │   │   )                                                                              │
│    71 │   │                                                                                      │
│    72 │   │   config = values["config"] or {}                                                    │
│ ❱  73 │   │   values["client"] = AutoModelForCausalLM.from_pretrained(                           │
│    74 │   │   │   values["model"],                                                               │
│    75 │   │   │   model_type=values["model_type"],                                               │
│    76 │   │   │   model_file=values["model_file"],                                               │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/ctransformers/hub.py:17 │
│ 5 in from_pretrained                                                                             │
│                                                                                                  │
│   172 │   │   │   │   revision=revision,                                                         │
│   173 │   │   │   )                                                                              │
│   174 │   │                                                                                      │
│ ❱ 175 │   │   llm = LLM(                                                                         │
│   176 │   │   │   model_path=model_path,                   

In [6]:
def get_langchain_chroma(device, persist_dir="../chroma_store"):
    model = PubMedBert(device=device)
    embed_fn = PubMedEmbeddingFunction(model=model)
    client = chromadb.PersistentClient(path=persist_dir)
    langchain_chroma = Chroma(
        client=client,
        collection_name="pubmed_embeddings",
        embedding_function=embed_fn,
        collection_metadata={"hnsw:space": "cosine"},
    )
    return langchain_chroma

In [22]:
class PubMedBert:
    def __init__(self, device):
        self.device = device
        self.model = SentenceTransformer(
            "pritamdeka/S-PubMedBert-MS-MARCO", device=self.device
        )
        self.model.max_seq_length = 512

    def encode(self, doc_batch):
        batch_size = len(doc_batch)
        embeddings = self.model.encode(
            doc_batch, device=self.device, batch_size=batch_size
        )
        return np.stack(embeddings, axis=0).tolist()


class PubMedEmbeddingFunction(chromadb.EmbeddingFunction):
    def __init__(self, model):
        self.model = model
        
    def embed_query(self, input):
        return self.model.encode([input])[0]

    def __call__(self, input):
        return self.model.encode(input)


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
vectordb = get_langchain_chroma(device=device)

cpu


In [24]:
#check if chroma got documents
print(vectordb._collection.count())

187838


In [25]:
question = "What is soft sensing?"

docs = vectordb.similarity_search(question,k=5)

In [19]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [20]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [21]:
# Pass question to the qa_chain
question = "What are important functions of electric skin?"
result = qa_chain({"query": question})
result["result"]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # Pass question to the qa_chain                                                              │
│   2 question = "What are important functions of electric skin?"                                  │
│ ❱ 3 result = qa_chain({"query": question})                                                       │
│   4 result["result"]                                                                             │
│   5                                                                                              │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/langchain/chains/base.p │
│ y:312 in __call__                                                                                │
│                                                                                                  │
│   309 │   │   │   )                                                                              │
│   310 │   │   except BaseException as e:                                                         │
│   311 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 312 │   │   │   raise e                                                                        │
│   313 │   │   run_manager.on_chain_end(outputs)                                                  │
│   314 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   315 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/langchain/chains/base.p │
│ y:306 in __call__                                                                                │
│                                                                                                  │
│   303 │   │   )                                                                                  │
│   304 │   │   try:                                                                               │
│   305 │   │   │   outputs = (                                                                    │
│ ❱ 306 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   307 │   │   │   │   if new_arg_supported                                                       │
│   308 │   │   │   │   else self._call(inputs)                                                    │
│   309 │   │   │   )                                                                              │
│                                                                                                  │
│ /Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/langchain/chains/retrie │
│ val_qa/base.py:144 in _call                                                                      │
│                                                                                                  │
│   141 │   │   │   docs = self._get_docs(question, run_manager=_run_manager)                      │
│   142 │   │   else:                                                                              │
│   143 │   │   │   docs = self._get_docs(question)  # type: ignore[call-arg]                      │
│ ❱ 144 │   │   answer = self.combine_documents_chain.run(                                         │
│   145 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   146 │   │   )                                            